In [1]:
import wbdata
import pandas as pd
import re
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pickle
import random
import time
from itertools import chain
path = 'c:/Users/silas/MIDS/jupyternotebooks/USAID/'

> Freedom House

In [2]:
# read in freedom house (fh) data and copy to data frame
fh_df = pd.read_csv('{}FH_Country_and_Territory_Ratings_and_Statuses_1972-2016.csv'.format(path))
fh_df.to_pickle('{}fh_df.pickle'.format(path))

In [3]:
fh_df.shape

(207, 133)

> U.S. Foreign Aid

In [4]:
# read in foreign aid (fa) data and copy to data frame
df_fa = pd.read_csv('{}us_foreign_aid_complete (1).csv'.format(path), low_memory=False)
df_fa.to_pickle('{}df_fa.pickle'.format(path))

In [5]:
df_fa.shape

(894897, 50)

> World Bank Data

In [6]:
# wbdata.get_source()
# wbdata.get_dataframe(indicators, country=u'all', data_date=None, convert_date=False, keep_levels=False)
# wbdata.get_data(indicator, country=u'all', data_date=None, convert_date=False, pandas=False, column_name=u'value', keep_levels=False)
 # wbdata.get_indicator(source=43)
'''
VA.EST          	Voice and Accountability: Estimate
VA.STD.ERR      	Voice and Accountability: Standard Error
NY.GDP.TOTL.RT.ZS	Total natural resources rents (% of GDP)
'''

# wb data import (natural resource rents, voice, pop, gdppc)
indicators = {"NY.GDP.TOTL.RT.ZS": "nat", "VA.EST": "voice", "NY.GDP.PCAP.PP.KD": "gdppc", "SP.POP.TOTL": "pop", "SI.POV.GINI": 'gini'}
wb_data_df = wbdata.get_dataframe(indicators, convert_date=True)
wb_data_df.reset_index(inplace=True)
wb_data_df['year'] = wb_data_df.date.dt.year
with open("{}wb_data_df.pickle".format(path), "wb") as f:
    pickle.dump(wb_data_df, f)

# wb voice indicator significantly inversely correlated with natural resource rents
wb_data_df.nat.corr(wb_data_df['voice'])
wb_codes_df = pd.read_csv('{}wb_codes_raw.csv'.format(path), header=None, names=['country']).country.str.split('     ', expand=True)

wb_codes_df = wb_codes_df.rename(columns={0:'wb_code',1:'country' })

wb_codes_df.country[-1:] = 'Zimbabwe'


with open("{}wb_codes_df.pickle".format(path), "wb") as f:
    pickle.dump(wb_codes_df, f)

> FIPS Codes

In [7]:
fips_html = pd.read_html('https://www.cia.gov/library/publications/the-world-factbook/appendix/appendix-d.html')

lst_fips = []
for i in range(3,len(fips_html)):
    lst_fips.append(tuple(fips_html[i].loc[0]))

fips_df = pd.DataFrame(lst_fips)

del fips_df[8]

fips_df = fips_df.dropna(subset=[3]).reset_index(drop=True)

fips_df.rename(columns={0:'country', 1:'GEC', 2:'not', 3:'ISO3166_S', 4:'ISO3166_C', 5:'ISO3166_N', 6:'STANAG', 7:'internet'}, inplace=True)

del fips_df['not']

with open("{}fips_df.pickle".format(path), "wb") as f:
    pickle.dump(fips_df, f)